In [1]:
# Sequence to Sequence <Seq2Seq>
# RNN 신경망과 출력하는 신경망을 조합한 모델
# 번역이나 챗봇등에서 많이 사용된다.
# 구글 기계번역에서도 이것을 사용하고 있다.

# Seq2Seq 모델은 크게 두가지로 구성된다.
# 1) 인코더 : 입력을 위한 신경망. 원문을 입력으로 받음.
# 2) 디코더 : 출력을 위한 신경망. 인코더가 번역한 결과물을 입력으로 받음.

In [2]:
# Seq2Seq 심볼은 특수한 심볼이 몇개 필요하다. 
# (골빈해커책의 195 페이지 참조)
# S 심볼 : 디코더에 입력이 시작됨을 알려주는 심볼
# E 심볼 : 디코더의 출력이 끝났음을 알려주는 심볼
# P 심볼 : 빈 데이터를 채울때 사용하는 무의미 심볼

import tensorflow as tf
import numpy as np

# 이제 데이터를 만들어보자. 먼저 영어알파벳과 한글들을 나열한 뒤에 한글자씩 배열에 집어넣어보자.

char_arr = [charcter for charcter in 'SEPabcdefghijklmnopqrstuvwxyz단어나무놀이소녀키스사랑']
num_dic = {n: i for i , n in enumerate(char_arr)}
dic_len = len(num_dic)

seq_data = [
    ['word', '단어'],
    ['wood', '나무'],
    ['game', '놀이'],
    ['girl', '소녀'],
    ['kiss', '키스'],
    ['love', '사랑']
]

In [3]:
# 그다음 입력 단어와 출력 단어를 한글자씩 떼어내서 배열로 만든 다음,
# 원-핫 인코딩 형태로 바꿔주는 함수를 만든다.
# 리턴 데이터는 아래와 같이 3가지로 구성된다.
# 1) 인코더의 입력값
# 2) 디코더의 입력값
# 3) 디코더의 출력값

def MakeBatch(_seqData):
    input_batch = []
    output_batch = []
    target_batch = []
    
    for iterSeq in _seqData:
        # 인코더 입력값 : 입력단어('word')를 한글자씩 떼어서 인덱스 배열로 만든다.
        inputData = [num_dic[n] for n in seq[0]]
        # 디코더 입력값 : 출력단어('단어') 앞에 디코더 시작을 나타내는 S 글자를 붙인다.
        # 그 후 한글자씩 떼어서 인덱스 배열로 만든다.
        outputData = [num_dic[n] for n in ('S' + seq[1])]
        # 디코더 출력값 : 출력단어('단어') 마지막에 디코더 끝을 알리는 심볼 'E' 글자를 붙인다.
        # 그 후 한글자씩 떼어서 인덱스 배열로 만든다.
        targetData = [num_dic[n] for n in (seq[1] + 'E')]
        
        # 인코더 입력값, 디코더 입력값은 원-핫 인코딩 형태로 인코딩하여 리스트에 추가하고,
        # 디코더 출력값만 정수 인덱스 값 그대로 리스트에 추가한다.
        input_batch.append(np.eye(dic_len)[inputData])
        output_batch.append(np.eye(dic_len)[outputData])
        target_batch.append(targetData)
    
    return input_batch, output_batch, target_batch       

In [ ]:
# 하이퍼 파라미터의 정의

hp_learning_rate = 0.01
hp_n_hidden = 128
hp_total_epoch = 100

hp_n_class = hp_n_input = dic_len

# 이제 플레이스홀더를 만들어보고자 한다.
# 여기에서 인코더 및 디코더의 입력값 형식은 아래와 같다.
# [batchSize, timeSteps, inputSize]

# 그리고 디코더 출력값의 형식은 아래와 같다.
# [batchSize, timeSteps]